In [1]:
%matplotlib inline
!pip install torch-geometric \
  torch-sparse==latest+cu101 \
  torch-scatter==latest+cu101 \
  torch-cluster==latest+cu101 \
  -f https://pytorch-geometric.com/whl/torch-1.5.0.html

Looking in links: https://pytorch-geometric.com/whl/torch-1.5.0.html
     |████████████████████████████████| 133kB 6.5MB/s 
     |████████████████████████████████| 22.0MB 1.2MB/s 
     |████████████████████████████████| 12.3MB 43.7MB/s 
     |████████████████████████████████| 18.2MB 169kB/s 
     |████████████████████████████████| 235kB 17.4MB/s 
     |████████████████████████████████| 51kB 7.3MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-1.4.3-cp36-none-any.whl size=234873 sha256=e64054dbb565f9fdd9a69cf94315abda88e34325e8559aef990338e59d3b489c
  Stored in directory: /root/.cache/pip/wheels/e2/c1/09/8693feee3f97e440d68b09abfca8b4c1e97150ace350b5003f
Successfully built torch-geometric


!pip install torch-geometric \
  torch-sparse==latest+cu101 \
  torch-scatter==latest+cu101 \
  torch-cluster==latest+cu101 \
  -f https://pytorch-geometric.com/whl/torch-1.5.0.html


In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import os

import time
import math

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader, InMemoryDataset

import numpy as np
from numpy import isnan
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from datetime import datetime


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('running on '+ str(device))

running on cuda


In [3]:
from google.colab import drive
import os.path
from os import path
import sys
from os import listdir
from os.path import isfile, join

drive.flush_and_unmount()
drive.mount('/content/drive')
PATH = '/content/drive/My Drive/rubber'
INPUT_ROOT = PATH+'/input_mix'
sys.path.append(PATH)

Drive not mounted, so nothing to flush and unmount.
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
class Seq2SeqDataSet(InMemoryDataset):
    def __init__(self, root, input_sequence, output_sequence, transform=None, pre_transform=None):
        super(Seq2SeqDataSet, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_dir(self):
      if os.path.exists(self.root+PROCESSED_DIR):
        return self.root+CLEANED_DIR #'/cleaned'
      else:
        os.mkdir(self.root+PROCESSED_DIR)
        return self.root+CLEANED_DIR#'/cleaned'
        
    @property
    def processed_dir(self):
      if os.path.exists(self.root+PROCESSED_DIR):
        return self.root+PROCESSED_DIR
      else:
        os.mkdir(self.root+PROCESSED_DIR)
        return self.root+PROCESSED_DIR

    @property
    def raw_file_names(self):
      mypath = self.raw_dir
      filenames = [f for f in listdir(mypath) if isfile(join(mypath, f))]
      return filenames

    @property
    def processed_file_names(self):
        return ['processed.dt']

    def download(self):
        pass
    
    def process(self):
      output_size , output_sequence_len, input_sequence_len= configuration['output_size'],\
      configuration['output_sequence_len'],\
      configuration['input_sequence_len'],
        
      data_list = []

      for raw_path in self.raw_paths:
        print(raw_path)
        df = pd.read_csv(raw_path)
        for synthetic_seq in df['synthesis_seq'].unique():
          synthetic_data = df[df['synthesis_seq']==synthetic_seq]

          for country in synthetic_data['countryterritoryCode'].unique():
            country_data = synthetic_data[synthetic_data['countryterritoryCode'] == country]
                        
            del country_data['synthesis_seq']
            del country_data['countryterritoryCode']

            country_data_i = country_data[:-output_sequence_len]
            country_data_array = country_data_i.to_numpy()
            feature_length = len(country_data.columns)
            x = country_data_array#[:feature_length].T

            country_data_o = country_data[input_sequence_len:]
            #country_data_array = country_data_array.reshape(feature_length,len(country_data_i))
            country_data_array_y = np.array([country_data_o['NATURAL_RUBBER_EXPORT'].to_numpy()])
            country_data_array_y = country_data_array_y.reshape(output_size,len(country_data_o))
            y = country_data_array_y[:output_size].T

            #print(country_data_array.shape)
            #print(country_data_array[0])
            
            
            #print(x.shape)
            sets =0
            x_list = []
            dict_x = dict()
            for i in range(input_sequence_len):
              array_len = (len(x)-i) - ((len(x)  - i)%input_sequence_len)+i
              if array_len <= 0:
                print('skipping')
                continue
              sets = int( array_len/ input_sequence_len)
              if sets <= 0:
                print('skipping')
                continue
              #print(len(x))
              #print('input seq : ', i , ' ', array_len , ' ',array_len-i , ' number of sets : ', sets)
              x_temp = x[i:array_len].T.reshape(sets, feature_length, input_sequence_len)
              uniq_keys = np.array([i+k*input_sequence_len for k in range(sets)])
              x_temp = x_temp.reshape(feature_length,sets,input_sequence_len)
              arrays_split = np.hsplit(x_temp,sets)
              dict_x.update(dict(zip(uniq_keys, arrays_split)))

            dict_y = dict()
            y_list = []
            for i in range(output_sequence_len):
              array_len_y = (len(y)-i) - ((len(y)  - i)%output_sequence_len)+i
              if array_len_y <= 0:
                continue
              sets = int(array_len_y / output_sequence_len)
              if sets <= 0:
                continue
              y_temp = y[i:array_len_y].T.reshape(sets, output_size, output_sequence_len)
              #uniq_keys = np.array([i+(output_sequence_len*k) for k in range(output_sequence_len)])
              uniq_keys = np.array([i+k*output_sequence_len for k in range(sets)])
              y_temp = y_temp.reshape(output_size,sets,output_sequence_len)
              arrays_split = np.hsplit(y_temp,sets)
              dict_y.update(dict(zip(uniq_keys, arrays_split)))

            temp_x_list = []
            mean = np.mean(country_data.to_numpy(), axis=0).T
            std = np.std(country_data.to_numpy(), axis=0).T
            #print(sorted(dict_x.keys()))
            for i in sorted(dict_x.keys()):
              x = dict_x[i].squeeze()
              #print(x.T[0][12])
              x = (x.T - mean) / std
              where_are_NaNs = isnan(x)
              x[where_are_NaNs] = 0
              temp_x_list.append(x)

            temp_y_list  = [dict_y[i].T for i in sorted(dict_y.keys())]

            #_country_code,popData2018
            xy_list = [Data(x = torch.from_numpy(features).type(torch.FloatTensor).squeeze()) for features in temp_x_list]
            
            mean_y = np.mean(y)
            std_y = np.std(y)

            if configuration['output_sequence_len'] > 1 :
              for i in sorted(dict_y.keys()):
                temp_y_list[i] = temp_y_list[i].squeeze()
                norm_y = (temp_y_list[i] -mean_y)/std_y
                xy_list[i].y = torch.from_numpy(norm_y).squeeze()
            else:
              for i in sorted(dict_y.keys()):
                #print(type(temp_y_list[i].squeeze()))
                #print(temp_y_list[i].squeeze())
                norm_y = (temp_y_list[i] -mean_y)/std_y
                xy_list[i].y = torch.from_numpy(norm_y).squeeze(1).squeeze(0)

            data_list += xy_list
        print('processed : '+ raw_path)
      data, slices = self.collate(data_list)
      torch.save((data, slices), self.processed_paths[0])

In [5]:
configuration = {
    'input_sequence_len' : 6,
    'output_sequence_len' : 1,
    'training_batch_size' : 2048,
    'training_dataset_length' :32768,
    'validation_batch_size' : 1024,
    'yhat_size' : 1,
    'feature_len' : 21,
    'output_size' : 1,
}


drive.flush_and_unmount()
drive.mount('/content/drive')

INPUT_ROOT = PATH+'/input_mix'
CLEANED_DIR = '/cleaned'
PROCESSED_DIR = '/processed_seq2seq_'+str(configuration['input_sequence_len'])+'_'+str(configuration['output_sequence_len'])
#os.rmdir(INPUT_ROOT+'/validation/'+PROCESSED_DIR)

validation_dataset = Seq2SeqDataSet(INPUT_ROOT+'/validation', configuration['input_sequence_len'], configuration['output_sequence_len'])
validation_dataset = validation_dataset.shuffle()
validation_dataloader = DataLoader(validation_dataset,batch_size=configuration['validation_batch_size'])
print(validation_dataset[0])

Mounted at /content/drive
Data(x=[6, 21], y=[1])


In [0]:
class RNNModel(nn.Module):
    def __init__(self, input_size, parameter_sizes, repeats ,output_size):
        super(RNNModel, self).__init__()
        self.input_size = input_size
        self.repeater_input_size = parameter_sizes[0]
        self.hidden_size = parameter_sizes[1]
        self.repeats = repeats
        self.output_size = output_size

        self.fc1 = nn.Linear(input_size, self.repeater_input_size)
        self.relu_activation = nn.ReLU()

        self.layers = dict()

        
        k = 0
        for i in range(repeats):
          i = i+k
          self.layers['fc_'+str(i)] = nn.Linear(self.repeater_input_size, self.hidden_size)
          self.layers['gru_'+str(i+1)] = nn.GRU(self.hidden_size, self.hidden_size)
          self.layers['fc_'+str(i+2)] = nn.Linear(self.hidden_size, self.repeater_input_size)
          k+=2

        self.module_list = nn.ModuleDict(self.layers)

        self.fc2 = nn.Linear(self.repeater_input_size, output_size)
        
    def forward(self, input, hidden):
      output = self.fc1(input)
      output = self.relu_activation(output)

      k = 0
      for i in range(self.repeats):
        i = i+k
        output = self.layers['fc_'+str(i)](output)
        output = self.relu_activation(output)

        self.layers['gru_'+str(i+1)].flatten_parameters()
        output, hidden[i-k] = self.layers['gru_'+str(i+1)](output, hidden[i-k])#should be different. check the nlp page
        output = self.relu_activation(output)
        hidden[i-k] = self.relu_activation(hidden[i-k])

        output = self.layers['fc_'+str(i+2)](output)
        output = self.relu_activation(output)
        k +=2

      output = self.fc2(output)
      #output = self.relu_activation(output)

      return output, hidden

    def initHidden(self, batch_size):
      hidden = torch.zeros(1, batch_size, self.hidden_size, device=device)
      return hidden

In [0]:
class DecoderNet(nn.Module):
    def __init__(self, input_size, parameter_sizes, repeats ,output_size):
        super(DecoderNet, self).__init__()
        self.input_size = input_size
        self.repeater_input_size = parameter_sizes[0]
        self.hidden_size = parameter_sizes[1]
        self.repeats = repeats
        self.output_size = output_size

        self.fc1 = nn.Linear(input_size, self.repeater_input_size)
        self.relu_activation = nn.ReLU()

        self.layers = dict()
        
        k = 0
        for i in range(repeats):
          i = i+k
          self.layers['fc_'+str(i)] = nn.Linear(self.repeater_input_size, self.hidden_size)
          self.layers['gru_'+str(i+1)] = nn.GRU(self.hidden_size, self.hidden_size)
          self.layers['fc_'+str(i+2)] = nn.Linear(self.hidden_size, self.repeater_input_size)
          k+=2

        self.module_list = nn.ModuleDict(self.layers)

        self.fc2 = nn.Linear(self.repeater_input_size, output_size)
        
    def forward(self, input,hidden):
      output = self.fc1(input)
      output = self.relu_activation(output)

      k = 0
      for i in range(self.repeats):
        i = i+k
        output = self.layers['fc_'+str(i)](output)
        output = self.relu_activation(output)
        self.layers['gru_'+str(i+1)].flatten_parameters()
        output, hidden[i-k] = self.layers['gru_'+str(i+1)](output, hidden[i-k])#should be different. check the nlp page
        output = self.relu_activation(output)
        hidden[i-k] = self.relu_activation(hidden[i-k])

        output = self.layers['fc_'+str(i+2)](output)
        output = self.relu_activation(output)
        k +=2

      output = self.fc2(output)
      
      return output, hidden

#    def initHidden(self, batch_size):
#        return torch.zeros(1, batch_size, self.hidden_size, device=device)

In [0]:
def train_rnn(input_tensor, target_tensor, rnn_model, rnn_model_optimizer, decoder_model, decoder_model_optimizer,  criterion,batch_size):
  #pass through the encoder
  input_tensor = input_tensor.type(torch.FloatTensor)
  target_tensor= target_tensor.type(torch.FloatTensor)
  input_tensor_length = len(input_tensor)
  
  rnn_model_hidden = [] #this status will be passed to the decoder
  for i in range(rnn_model.repeats):
    rnn_model_hidden.append(rnn_model.initHidden(batch_size))

  encoder_outputs = torch.zeros([configuration['input_sequence_len'],input_tensor_length], dtype=torch.float32).to(device)

  for ei in range(configuration['input_sequence_len']):
    #target_tensor_seq = target_tensor.view(configuration['output_sequence_len'], batch_size, configuration['yhat_size'])
    input_tensor_seq = input_tensor.view(configuration['input_sequence_len'],batch_size, configuration['feature_len'])[ei]
    input_tensor_seq = input_tensor_seq.view(1, batch_size, configuration['feature_len']).to(device)
    rnn_model_output, rnn_model_hidden = rnn_model(
        input_tensor_seq, rnn_model_hidden)
    encoder_outputs[ei]= rnn_model_output.squeeze()

  #pass through the decoder  
  loss = 0
  decoder_model_optimizer.zero_grad()
  rnn_model_optimizer.zero_grad()
  encoder_outputs = torch.transpose(encoder_outputs, 0, 1).unsqueeze(0)
  
  decoder_output, decoder_hidden = decoder_model(encoder_outputs, rnn_model_hidden)
  target_tensor_seq= target_tensor.view(configuration['output_sequence_len'], batch_size, configuration['yhat_size'])
  loss += criterion(decoder_output.squeeze(), target_tensor_seq.squeeze().to(device))
  loss.backward()

  rnn_model_optimizer.step()
  decoder_model_optimizer.step()
  

  return loss.item() / configuration['input_sequence_len']

In [0]:
def evaluate_rnn(input_tensor, target_tensor, rnn_model, rnn_model_optimizer, decoder_model, decoder_optimizer, criterion,batch_size):
  with torch.no_grad():
    #pass through the encoder
    input_tensor = input_tensor.type(torch.FloatTensor)
    target_tensor= target_tensor.type(torch.FloatTensor)
    input_tensor_length = len(input_tensor)
    
    rnn_model_hidden = [] #this status will be passed to the decoder
    for i in range(rnn_model.repeats):
      rnn_model_hidden.append(rnn_model.initHidden(batch_size))

    encoder_outputs = torch.zeros([configuration['input_sequence_len'],input_tensor_length], dtype=torch.float32).to(device)

    for ei in range(configuration['input_sequence_len']):
      #target_tensor_seq = target_tensor.view(configuration['output_sequence_len'], batch_size, configuration['yhat_size'])
      input_tensor_seq = input_tensor.view(configuration['input_sequence_len'],batch_size, configuration['feature_len'])[ei]
      input_tensor_seq = input_tensor_seq.view(1, batch_size, configuration['feature_len']).to(device)
      rnn_model_output, rnn_model_hidden = rnn_model(
          input_tensor_seq, rnn_model_hidden)
      encoder_outputs[ei]= rnn_model_output.squeeze()

    #pass through the decoder  
    validation_loss = 0
    encoder_outputs = torch.transpose(encoder_outputs, 0, 1).unsqueeze(0)
    decoder_output, decoder_hidden = decoder_model(encoder_outputs, rnn_model_hidden)
    target_tensor_seq= target_tensor.view(configuration['output_sequence_len'], batch_size, configuration['yhat_size'])
    validation_loss += criterion(decoder_output.squeeze(), target_tensor_seq.squeeze().to(device))
      
    return validation_loss.item() / configuration['input_sequence_len']

In [13]:
INPUT_ROOT = PATH+'/input_mix'
CLEANED_DIR = '/cleaned'
#PROCESSED_DIR = '/processed_seq2seq_'+str(configuration['input_sequence_len'])+'_'+str(configuration['input_sequence_len'])
PROCESSED_DIR = "processed_seq2seq_6_1"

test_dataset = Seq2SeqDataSet(INPUT_ROOT+'/validation', configuration['input_sequence_len'], configuration['input_sequence_len'])
test_dataset = test_dataset.shuffle()
test_dataset = test_dataset[:configuration['training_dataset_length']]
test_dataloader = DataLoader(test_dataset,batch_size=configuration['training_batch_size'])
training_dataloader = test_dataloader
validation_dataloader = test_dataloader

#load the latest model for training
rnn_model = RNNModel(input_size=21,parameter_sizes=[256,256],repeats=1,output_size=1).to(device)#get_latest_model(PATH+'/models_rnn')
decoder_model = DecoderNet(input_size=6,parameter_sizes=[256,256],repeats=1,output_size=1).to(device)
rnn_model_optimizer = optim.SGD(rnn_model.parameters(), lr=0.1, momentum=0.9)
decoder_model_optimizer = optim.SGD(decoder_model.parameters(), lr=0.1, momentum=0.9)

train_models( rnn_model, rnn_model_optimizer , decoder_model, decoder_model_optimizer ,learning_rate = 0.1, epochs = 1,id= str(1) , validation=False)

NameError: ignored

In [14]:
def generate_repeat_counts(alphas , training_dataset_length,
                           enc_repeated_nn_parameters = 132096):
  repeats = []
  feature_len = configuration['feature_len']
  yhat_size = configuration['output_size']
  model1 = RNNModel(**{
         'input_size': configuration['feature_len'],
         'parameter_sizes': [256, 256] ,
         'repeats' : 1 ,
         'output_size' : yhat_size
        })
  model2 = RNNModel(**{
         'input_size': configuration['feature_len'],
         'parameter_sizes': [256, 256] ,
         'repeats' : 2 ,
         'output_size' : yhat_size
        }
  )

  enc_repeated_nn_parameters = get_n_params(model2) - get_n_params(model1)
  print(enc_repeated_nn_parameters)

  for alpha in alphas:
    preferred_parameters  = (1/alpha)* (training_dataset_length / (configuration['feature_len']+configuration['yhat_size']))
    enc_repeats = int(
        (preferred_parameters - 
        (preferred_parameters%enc_repeated_nn_parameters))
        / enc_repeated_nn_parameters)
    for j in range(1, enc_repeats+1):
      if j not in repeats:
        repeats.append(j)
  return repeats

def generate_model_parameters(repeat_counts):
  model_dict = dict()
  model_id = 0
  feature_len = configuration['feature_len']
  output_size = configuration['output_size']

  for enc_repeat in repeat_counts:
    model_dict[model_id] = [{
         'input_size': feature_len,
         'parameter_sizes': [256, 256] ,
         'repeats' : enc_repeat ,
         'output_size' : output_size
        }
        ]
    model_id+=1
  return model_dict

def save_model_parameters(model_parameters_dict, models = None, fileName=None):
  if fileName == None:
    fileName = 'model_parameters_'+str(np.round(np.random.randn(1000,200)[0][0]*1000))

  model_summary = pd.DataFrame(columns =[])
  for i in sorted(model_parameters_dict.keys()):
    param_size = 0
    if models != None:
      param_size = get_n_params(models[i][0])
      param_size += get_n_params(models[i][1])
    row = pd.Series({
                        'model_id':i,
                        'repeats' : model_parameters_dict[i][0]['repeats'],
                        'parameters' :param_size
    })
    row_df = pd.DataFrame([row], index = [i])
    model_summary = pd.concat([model_summary, row_df])
  model_summary.to_csv(fileName)

#Create a new directory to store model results
MODEL_PREFIX = '_rnn'
MODEL_DIR =  PATH + '/models'+MODEL_PREFIX
dirs = []
for f in listdir(MODEL_DIR):
  if "run" in f:
    dirs.append(int(f[3]))
if len(dirs) > 0:
  if os.path.exists(MODEL_DIR+"/run"+str(max(dirs)+1)) == False:
    os.mkdir(MODEL_DIR+"/run"+str(max(dirs)+1))
    MODEL_DIR = MODEL_DIR+"/run"+str(max(dirs)+1)
else:
  os.mkdir(MODEL_DIR+"/run0")
  MODEL_DIR = MODEL_DIR+"/run0"

if os.path.exists(MODEL_DIR)==False:
  os.mkdir(MODEL_DIR)

session_id =str(1001)+ '_'+str(datetime.now(tz=None).microsecond)
repeat_counts = generate_repeat_counts([1,2,3], 100000000)
print(repeat_counts)

model_configurations = generate_model_parameters(repeat_counts)
save_model_parameters(model_configurations, fileName= MODEL_DIR + '/rnn_model_configurations_'+session_id+'.csv')
models = dict()
print(len(model_configurations))
print(len(repeat_counts))

526336
[1, 2, 3, 4, 5, 6, 7, 8]
8
8


In [0]:
#Run all the candidate models
def train_models( rnn_model, rnn_model_optimizer, decoder_model, decoder_model_optimizer,learning_rate = 0.001, epochs = 2500, id= "0",  validation=True):
  for i in model_configurations:
    #instantiate the models
    if i != 1 :
      print('skipping', i)
      continue
    current_model_dir =  MODEL_DIR +'/'+str(i)
    if os.path.exists(current_model_dir) == False:
      os.mkdir(current_model_dir)
    
    criterion = nn.MSELoss()

    losses ,rnn_model , rnn_model_optimizer, decoder_model, decoder_model_optimizer = trainValidationIters(
        training_dataloader,validation_dataloader,
        rnn_model,rnn_model_optimizer, 
        decoder_model,decoder_model_optimizer,
        criterion, n_iters = epochs,output_sequence_len=configuration['output_sequence_len'],
        input_sequence_len=configuration['input_sequence_len'],
        current_model_dir=current_model_dir,  model_id = i,session_id = session_id,
        print_every=25, save_every = 100,check_validation_every=50, id=id, validation=validation)
    
    print('Model ',str(i), ': trained and evaluated')
    break
    
    #save_dict({
    #    'model' : rnn_model.state_dict(),
    #    'losses' : losses
    #}, current_model_dir+'/'+str(i)+'models_'+session_id+'.pt')

    #models[i] = {
    #    'rnn_model' : rnn_model.state_dict(),
    #    'losses' : losses
    #}

  #save_dict(models, MODEL_DIR+'/models_'+session_id+'.pt')
  return rnn_model, rnn_model_optimizer, decoder_model, decoder_model_optimizer

In [16]:
#INPUT_ROOT = PATH+'/input_mix'
#CLEANED_DIR = '/cleaned'
#PROCESSED_DIR = '/processed_seq2seq_'+str(configuration['input_sequence_len'])+'_'+str(configuration['input_sequence_len'])

#validation_dataset = Seq2SeqDataSet(INPUT_ROOT+'/validation', configuration['input_sequence_len'], configuration['output_sequence_len'])
#validation_dataset = validation_dataset.shuffle()
#validation_dataloader = DataLoader(validation_dataset,batch_size=configuration['validation_batch_size'])
len(validation_dataset)

138

In [29]:
INPUT_ROOT = PATH+'/input_mix'
CLEANED_DIR = '/cleaned'
PROCESSED_DIR = '/processed_seq2seq_'+str(configuration['input_sequence_len'])+'_'+str(configuration['input_sequence_len'])

#validation_dataset = Seq2SeqDataSet(INPUT_ROOT+'/validation', configuration['input_sequence_len'], configuration['output_sequence_len'])
#validation_dataset = validation_dataset.shuffle()
#validation_dataloader = DataLoader(validation_dataset,batch_size=configuration['validation_batch_size'])
learning_rate = 0.001

rnn_model = RNNModel(input_size=21,parameter_sizes=[128,256],repeats=1,output_size=1).to(device)
decoder_model = DecoderNet(input_size=6,parameter_sizes=[128,256],repeats=1,output_size=1).to(device)
rnn_model_optimizer = optim.SGD(rnn_model.parameters(), lr=learning_rate, momentum=0.9)
decoder_model_optimizer = optim.SGD(decoder_model.parameters(), lr=learning_rate, momentum=0.9)

dirs = ['/'+f for f in listdir(INPUT_ROOT+'/training') if "cleaned_non_window" in f]
dirs.reverse()
print(dirs)

i = 0
for CLEANED_DIR  in dirs:
  PROCESSED_DIR = CLEANED_DIR.split( sep="_")[0]+'_seq2seq_' +str(configuration['input_sequence_len'])+'_'+str(configuration['output_sequence_len'])
  print(PROCESSED_DIR)
  
  training_dataset = Seq2SeqDataSet(INPUT_ROOT+'/training', configuration['input_sequence_len'], configuration['input_sequence_len'])
  training_dataset = training_dataset.shuffle()
  training_dataset = training_dataset[:configuration['training_dataset_length']]
  training_dataloader = DataLoader(training_dataset,batch_size=configuration['training_batch_size'])

  rnn_model, rnn_model_optimizer, decoder_model, decoder_model_optimizer =  train_models( 
      rnn_model,rnn_model_optimizer,
      decoder_model=decoder_model, decoder_model_optimizer=decoder_model_optimizer ,learning_rate = learning_rate, epochs = 1000,id= str(i) )
  
  torch.save(rnn_model, PATH+'/models_rnn'+CLEANED_DIR+'_rnn.model')
  torch.save(rnn_model_optimizer, PATH+'/models_rnn'+CLEANED_DIR+'_rnn.optim')
  torch.save(decoder_model, PATH+'/models_rnn'+CLEANED_DIR+'_decoder.model')
  torch.save(decoder_model_optimizer, PATH+'/models_rnn'+CLEANED_DIR+'_decoder.optim')
  i+=1

['/2000_cleaned_non_window', '/4000_cleaned_non_window', '/6000_cleaned_non_window', '/8000_cleaned_non_window', '/10000_cleaned_non_window', '/12000_cleaned_non_window', '/14000_cleaned_non_window', '/16000_cleaned_non_window', '/18000_cleaned_non_window', '/20000_cleaned_non_window', '/30000_cleaned_non_window', '/40000_cleaned_non_window', '/50000_cleaned_non_window', '/60000_cleaned_non_window', '/70000_cleaned_non_window', '/80000_cleaned_non_window', '/90000_cleaned_non_window', '/200000_cleaned_non_window', '/300000_cleaned_non_window', '/400000_cleaned_non_window', '/500000_cleaned_non_window', '/600000_cleaned_non_window', '/700000_cleaned_non_window', '/800000_cleaned_non_window', '/900000_cleaned_non_window', '/1000000_cleaned_non_window']
/2000_seq2seq_6_1
skipping 0
1m 51s (- 72m 23s) (25 2%) training:0.000082328511404 validation:0.001207872122382
train/validation difference :  0.00112554361097776
3m 43s (- 70m 39s) (50 5%) training:0.000082313730200 validation:0.001207922

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type RNNModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type DecoderNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


skipping 0
1m 51s (- 72m 23s) (25 2%) training:0.000080543564764 validation:0.001207734219694
train/validation difference :  0.001127190654930405
3m 42s (- 70m 28s) (50 5%) training:0.000080543564764 validation:0.001207734219694
train/validation difference :  0.001127190654930405
4.44331997762615e-20
[0.0012077342196939072, 0.0012077342196939072]
m :  4.44331997762615e-20
Terminating the training at : 50
Model  1 : trained and evaluated
/6000_seq2seq_6_1
skipping 0
1m 51s (- 72m 29s) (25 2%) training:0.000081915227687 validation:0.001207734219694
train/validation difference :  0.001125818992007309
3m 41s (- 70m 7s) (50 5%) training:0.000081915227687 validation:0.001207734219694
train/validation difference :  0.001125818992007309
4.44331997762615e-20
[0.0012077342196939072, 0.0012077342196939072]
m :  4.44331997762615e-20
Terminating the training at : 50
Model  1 : trained and evaluated
/8000_seq2seq_6_1
skipping 0
1m 50s (- 71m 44s) (25 2%) training:0.000081685108550 validation:0.00120

KeyboardInterrupt: ignored

In [0]:
validation_dataset[0]

In [0]:
import copy

def train_batch(dataloader, 
                rnn_model,rnn_model_optimizer, 
                decoder_model, decoder_model_optimizer,criterion):
  loss = 0
  for batch in dataloader:
    batch_size = int(len(batch.x)/configuration['input_sequence_len'])
    loss += (train_rnn(batch.x.view(batch_size, configuration['input_sequence_len'] , configuration['feature_len']), batch.y,
                       rnn_model,rnn_model_optimizer,
                       decoder_model, decoder_model_optimizer,
                       criterion, batch_size))/ batch_size
  return loss / len(dataloader)

def batch_evaluate(dataloader, 
                   rnn_model,rnn_model_optimizer,
                   decoder_model, decoder_model_optimizer, criterion, output_sequence_len):
  loss = 0
  for batch in dataloader:
    batch_size = int(len(batch.x)/configuration['input_sequence_len'])
    loss += (evaluate_rnn(batch.x.view(batch_size, configuration['input_sequence_len'] , configuration['feature_len']), batch.y,
                          rnn_model,rnn_model_optimizer,
                          decoder_model, decoder_model_optimizer,
                          criterion, batch_size))/ batch_size
  return loss / len(dataloader)

def trainValidationIters(train_dataloader,validation_dataloader,
                         rnn_model, rnn_model_optimizer,
                         decoder_model, decoder_model_optimizer, 
                        criterion ,n_iters, output_sequence_len,input_sequence_len,
                        current_model_dir, model_id,session_id,print_every=1000, plot_every=100,save_every = 100,check_validation_every=5,id="train_id_", validation=True):

  start = time.time()
  plot_losses = []
  
  # Reset every print_every # Reset every plot_every
  print_training_loss_total = 0
  plot_training_loss_total = 0
  print_validation_loss_total = 0
  plot_validation_loss_total = 0
  save_validation_loss_total = 0
  save_training_loss_total = 0
      
  losses = []
  rnn_models_temp = dict()
  rnn_optim_temp = dict()
  decoder_models_temp = dict()
  decoder_optim_temp = dict()
  for iter in range(1, n_iters + 1):
      decoder_model.train()
      rnn_model.train()
      training_loss = train_batch(train_dataloader, 
                                  rnn_model,rnn_model_optimizer,
                                  decoder_model, decoder_model_optimizer, criterion )
      
      if validation == True:
        rnn_model.eval()
        decoder_model.eval()
        validation_loss =  batch_evaluate(validation_dataloader, 
                                          rnn_model,rnn_model_optimizer,
                                          decoder_model, decoder_model_optimizer, 
                                          criterion,configuration['output_sequence_len'] )
      else:
        validation_loss = 0

      rnn_models_temp[validation_loss] = copy.deepcopy(rnn_model) #Save the model
      rnn_optim_temp[validation_loss] = copy.deepcopy(rnn_model_optimizer) #Save the model
      decoder_models_temp[validation_loss] = copy.deepcopy(decoder_model) #Save the model
      decoder_optim_temp[validation_loss] = copy.deepcopy(decoder_model_optimizer) #Save the model

      print_training_loss_total += training_loss
      plot_training_loss_total += training_loss
      print_validation_loss_total += validation_loss
      plot_validation_loss_total += validation_loss
      
      save_training_loss_total += training_loss
      save_validation_loss_total += validation_loss
      
      if iter % print_every == 0:
          print_training_loss_avg = print_training_loss_total / print_every
          print_training_loss_total = 0
          print_validation_loss_avg = print_validation_loss_total / print_every
          print_validation_loss_total = 0
          print('%s (%d %d%%) training:%.15f validation:%.15f' % (timeSince(start, iter / n_iters),
                                        iter, iter / n_iters * 100, print_training_loss_avg,print_validation_loss_avg, ))
          print('train/validation difference : ',print_validation_loss_avg-print_training_loss_avg)
          current_model = rnn_model
          current_optimizer = decoder_model_optimizer
        
      if iter % save_every == 0:
        save_training_loss_avg = save_training_loss_total / save_every
        save_validation_loss_avg = plot_validation_loss_total / save_every
        #'arch': args.arch,
        save_checkpoint({
            'epoch': iter + 1,
            'rnn_model_state_dict': rnn_model.state_dict(),
            'training_loss': save_training_loss_avg,
            'validation_loss': save_validation_loss_avg,
        }, False, filename=current_model_dir+'/'+str(iter)+'_'+str(session_id)+'.pt')

      losses.append([training_loss, validation_loss])

      if iter % check_validation_every == 0 and iter >= check_validation_every:
        #x = [i for i in range(1,check_validation_every+1)]
        x = [0 , 1]
        y = np.array(losses).T[1][-1*check_validation_every:].tolist()
        y = [y[-1*check_validation_every:][0], y[-1*check_validation_every:][-1]]
        m,b = np.polyfit(x, y, 1)
        print(m)
        print(y)
        if m >= -1*10**-9 :
          print('m : ', m)
          print('Terminating the training at :',iter )
          break
  
  for k in sorted(rnn_models_temp.keys()):
    rnn_model = rnn_models_temp[k]
    rnn_model_optimizer = rnn_optim_temp[k]
    break

  for k in sorted(decoder_models_temp.keys()):
    decoder_model = decoder_models_temp[k]
    decoder_model_optimizer = decoder_optim_temp[k]
    break

  save_dict({
      'losses' : losses
  }, filename=current_model_dir+'/losses_'+id)

  return losses , rnn_model , rnn_model_optimizer, decoder_model, decoder_model_optimizer

In [0]:
#los = [1,2,3,4,5,5,6,7,87,9]
#print(los[-3:][-1])
#print(los[-3:][0])

In [0]:
"""
Common function
"""
def get_latest_model(mypath):
  model = RNNModel(**model_configurations[4][0])
  fnames= []
  for fname in listdir(mypath):
    if "cleaned_non_window" in fname:
      fnames.append(int(fname.split('_')[0]))
  print('Model path: ', mypath+'/'+str(min(fnames))+'_cleaned_non_window.model')

  if torch.cuda.is_available():
    print('cuda rnn_model')
    return torch.load(mypath+'/'+str(min(fnames))+'_cleaned_non_window.model')
  else:
    print('cpu rnn_model')
    return torch.load(mypath+'/'+str(min(fnames))+'_cleaned_non_window.model',  map_location=torch.device('cpu'))
  return model

def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp
  

def save_model_parameters(model_parameters_dict, models = None, fileName=None):
  if fileName == None:
    fileName = 'model_parameters_'+str(np.round(np.random.randn(1000,200)[0][0]*1000))

  model_summary = pd.DataFrame(columns =[])
  for i in sorted(model_parameters_dict.keys()):
    param_size = 0
    if models != None:
      param_size = get_n_params(models[i][0])
      param_size += get_n_params(models[i][1])
    row = pd.Series({
                        'model_id':i,
                        'encoder repeats' : model_parameters_dict[i][0]['repeats'],
                        'decoder repeats' : model_parameters_dict[i][1]['repeats'],
                        'parameters' :param_size
    })
    row_df = pd.DataFrame([row], index = [i])
    model_summary = pd.concat([model_summary, row_df])
  model_summary.to_csv(fileName)

#Generate the list of repeat count parameters list

#generate_model_parameters(generate_repeat_counts())

def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

def save_dict(dictionary, filename):
  np.save(filename, dictionary)

import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points, points2):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.xlabel('Iteration')
    plt.ylabel('Loss')
    plt.plot(points)
    plt.plot(points2)

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))
